# CTI-BERT TTP Tagging - GPU Training Pipeline

Bu notebook, Google Colab GPU ile VS Code entegrasyonu üzerinden CTI-BERT modelini eğitmek için hazırlanmıştır.

**Dataset:** tumeteor/Security-TTP-Mapping (20,736 samples, 499 MITRE ATT&CK techniques)

---

## 🚀 Hızlı Başlangıç

1. **VS Code'da Colab bağlantısını kontrol et**
2. **GPU'nun aktif olduğundan emin ol** (Runtime → Change runtime type → GPU)
3. **Run All** veya hücreleri sırayla çalıştır

---

## ⚙️ Yapılandırma

Aşağıdaki parametreleri değiştirebilirsin:
- `MODEL_NAME`: "bert-base-uncased" veya "jackaduma/SecBERT"
- `NUM_EPOCHS`: 3 (varsayılan)
- `BATCH_SIZE`: 16 (GPU memory'ye göre ayarla)
- `LEARNING_RATE`: 2e-5 (varsayılan)

---

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("Running on CPU")

In [ ]:
# Import required libraries
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import torch.nn as nn
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from tqdm import tqdm
import numpy as np
import pandas as pd
import json
import ast
import os
from datetime import datetime

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Create output directory
output_dir = f"./outputs/colab_training_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory: {output_dir}")

## 1. Data Loading

In [ ]:
# Load dataset
print("Loading dataset...")
dataset = load_dataset("tumeteor/Security-TTP-Mapping")

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['validation'])  # Using validation as test

print(f"Train samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")
print(f"\nSample data:")
print(train_df.head(2))

In [ ]:
# Extract unique labels
def get_label_list(df):
    all_labels = set()
    for labels in df['labels']:
        label_list = ast.literal_eval(labels)
        all_labels.update(label_list)
    return sorted(list(all_labels))

label_list = get_label_list(train_df)
num_labels = len(label_list)
label_to_id = {label: idx for idx, label in enumerate(label_list)}

print(f"Total unique MITRE ATT&CK techniques: {num_labels}")
print(f"First 10 techniques: {label_list[:10]}")

In [ ]:
# Encode labels to binary vectors
def encode_labels(df):
    encoded = []
    for labels in df['labels']:
        label_vector = [0] * num_labels
        label_items = ast.literal_eval(labels)
        for label in label_items:
            if label in label_to_id:
                label_vector[label_to_id[label]] = 1
        encoded.append(label_vector)
    return encoded

train_labels = encode_labels(train_df)
test_labels = encode_labels(test_df)

print(f"Label encoding complete!")
print(f"Sample label vector shape: {len(train_labels[0])}")
print(f"Sample label vector (first 20): {train_labels[0][:20]}")

## 2. Tokenization

In [ ]:
# Load tokenizer
MODEL_NAME = "bert-base-uncased"  # Change to "jackaduma/SecBERT" for domain model
MAX_LENGTH = 512

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(f"Tokenizing texts...")
train_encodings = tokenizer(
    train_df['text1'].tolist(),
    truncation=True,
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors=None
)

test_encodings = tokenizer(
    test_df['text1'].tolist(),
    truncation=True,
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors=None
)

print("Tokenization complete!")

In [ ]:
# Create dataset
class CTIDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = CTIDataset(train_encodings, train_labels)
test_dataset = CTIDataset(test_encodings, test_labels)

print(f"Datasets created: {len(train_dataset)} train, {len(test_dataset)} test")

## 3. Model Definition

In [ ]:
class BERTForMultiLabelClassification(nn.Module):
    def __init__(self, model_name, num_labels, dropout=0.1):
        super().__init__()
        self.num_labels = num_labels
        self.bert = AutoModel.from_pretrained(model_name)
        config = AutoConfig.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        
        # Initialize classifier
        self.classifier.weight.data.normal_(mean=0.0, std=config.initializer_range)
        self.classifier.bias.data.zero_()
    
    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [CLS] token
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels)
        
        return {'loss': loss, 'logits': logits}

# Initialize model
model = BERTForMultiLabelClassification(MODEL_NAME, num_labels).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"Model loaded on {device}")
print(f"Total parameters: {total_params:,}")

## 4. Training

In [ ]:
# Training configuration
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_EPOCHS = 3
WARMUP_STEPS = 500

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, WARMUP_STEPS, total_steps)

print(f"Training configuration:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Total steps: {total_steps}")

In [ ]:
# Training function
def train_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    
    progress_bar = tqdm(dataloader, desc="Training")
    
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs['loss']
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})
    
    return total_loss / len(dataloader)

# Train the model
history = {'train_loss': [], 'epoch': []}

print("\n" + "="*70)
print("STARTING TRAINING")
print("="*70)

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS}")
    print("-" * 50)
    
    avg_loss = train_epoch(model, train_loader, optimizer, scheduler, device)
    
    print(f"Average training loss: {avg_loss:.4f}")
    
    history['train_loss'].append(avg_loss)
    history['epoch'].append(epoch + 1)

print("\nTraining complete!")

## 5. Evaluation

In [ ]:
# Prediction function
def predict(model, dataloader, device, threshold=0.5):
    model.eval()
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Predicting"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs['logits']
            predictions = torch.sigmoid(logits)
            predictions = (predictions > threshold).float()
            
            all_predictions.append(predictions.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    all_predictions = np.vstack(all_predictions)
    all_labels = np.vstack(all_labels)
    
    return all_predictions, all_labels

# Get predictions
predictions, labels = predict(model, test_loader, device)

In [ ]:
# Compute metrics
micro_f1 = f1_score(labels, predictions, average='micro', zero_division=0)
micro_precision = precision_score(labels, predictions, average='micro', zero_division=0)
micro_recall = recall_score(labels, predictions, average='micro', zero_division=0)

macro_f1 = f1_score(labels, predictions, average='macro', zero_division=0)
macro_precision = precision_score(labels, predictions, average='macro', zero_division=0)
macro_recall = recall_score(labels, predictions, average='macro', zero_division=0)

samples_f1 = f1_score(labels, predictions, average='samples', zero_division=0)
subset_accuracy = accuracy_score(labels, predictions)

print("\n" + "="*70)
print("EVALUATION RESULTS")
print("="*70)
print(f"\nMicro-averaged metrics (overall):")
print(f"  F1 Score:  {micro_f1:.4f}")
print(f"  Precision: {micro_precision:.4f}")
print(f"  Recall:    {micro_recall:.4f}")

print(f"\nMacro-averaged metrics (per label):")
print(f"  F1 Score:  {macro_f1:.4f}")
print(f"  Precision: {macro_precision:.4f}")
print(f"  Recall:    {macro_recall:.4f}")

print(f"\nOther metrics:")
print(f"  Samples F1:      {samples_f1:.4f}")
print(f"  Subset Accuracy: {subset_accuracy:.4f}")
print("="*70)

## 6. Save Results

In [ ]:
# Save model and results
model_path = os.path.join(output_dir, 'cti_bert_model.pt')
results_path = os.path.join(output_dir, 'results.json')
labels_path = os.path.join(output_dir, 'labels.json')

# Save model
torch.save(model.state_dict(), model_path)
print(f"Model saved: {model_path}")

# Save label list
with open(labels_path, 'w') as f:
    json.dump(label_list, f, indent=2)
print(f"Labels saved: {labels_path}")

# Save results
results = {
    'model': MODEL_NAME,
    'num_labels': num_labels,
    'train_samples': len(train_dataset),
    'test_samples': len(test_dataset),
    'metrics': {
        'micro_f1': float(micro_f1),
        'micro_precision': float(micro_precision),
        'micro_recall': float(micro_recall),
        'macro_f1': float(macro_f1),
        'macro_precision': float(macro_precision),
        'macro_recall': float(macro_recall),
        'samples_f1': float(samples_f1),
        'subset_accuracy': float(subset_accuracy)
    },
    'training_history': history,
    'hyperparameters': {
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'num_epochs': NUM_EPOCHS,
        'max_length': MAX_LENGTH,
        'warmup_steps': WARMUP_STEPS
    }
}

with open(results_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"Results saved: {results_path}")
print(f"\nAll outputs saved to: {output_dir}")

## 7. Visualization

In [ ]:
import matplotlib.pyplot as plt

# Training loss plot
plt.figure(figsize=(10, 6))
plt.plot(history['epoch'], history['train_loss'], marker='o', linewidth=2, markersize=8)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training Loss over Epochs', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
loss_plot_path = os.path.join(output_dir, 'training_loss.png')
plt.savefig(loss_plot_path, dpi=300, bbox_inches='tight')
print(f"Loss plot saved: {loss_plot_path}")
plt.show()

# Metrics comparison
metrics_names = ['Micro F1', 'Macro F1', 'Samples F1']
metrics_values = [micro_f1, macro_f1, samples_f1]

plt.figure(figsize=(10, 6))
bars = plt.bar(metrics_names, metrics_values, color=['#3498db', '#2ecc71', '#f39c12'], width=0.6)
plt.ylabel('Score', fontsize=12)
plt.title('Model Performance Metrics', fontsize=14, fontweight='bold')
plt.ylim([0, 1])
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, v in enumerate(metrics_values):
    plt.text(i, v + 0.02, f'{v:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
metrics_plot_path = os.path.join(output_dir, 'metrics_comparison.png')
plt.savefig(metrics_plot_path, dpi=300, bbox_inches='tight')
print(f"Metrics plot saved: {metrics_plot_path}")
plt.show()

# Precision-Recall comparison
pr_metrics = ['Precision', 'Recall', 'F1']
micro_values = [micro_precision, micro_recall, micro_f1]
macro_values = [macro_precision, macro_recall, macro_f1]

x = np.arange(len(pr_metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, micro_values, width, label='Micro-avg', color='#3498db')
bars2 = ax.bar(x + width/2, macro_values, width, label='Macro-avg', color='#e74c3c')

ax.set_xlabel('Metrics', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Precision, Recall, F1 Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(pr_metrics)
ax.legend(fontsize=11)
ax.set_ylim([0, 1])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
pr_plot_path = os.path.join(output_dir, 'precision_recall_comparison.png')
plt.savefig(pr_plot_path, dpi=300, bbox_inches='tight')
print(f"P-R comparison plot saved: {pr_plot_path}")
plt.show()

print(f"\n{'='*70}")
print("TRAINING COMPLETE!")
print(f"{'='*70}")
print(f"\nAll results saved to: {output_dir}")
print(f"\nFiles created:")
print(f"  - cti_bert_model.pt")
print(f"  - results.json")
print(f"  - labels.json")
print(f"  - training_loss.png")
print(f"  - metrics_comparison.png")
print(f"  - precision_recall_comparison.png")
print(f"{'='*70}")